In [1]:
import math
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt

# Main Functions



```
S_array = np.array(
    [
        [['x', 'y', 'z'],['x', 'y', 'z'],['x', 'y', 'z'], ... , ['x', 'y', 'z']], # N three dimensional spins
        
         ...

        [['x', 'y', 'z'],['x', 'y', 'z'],['x', 'y', 'z'], ... , ['x', 'y', 'z']]
        # No. of Ensemble
    ]
)
```



In [2]:
def calV_3D(N, ens, mc, S_array):
  '''
  We calculate the diversity of group opinions using the function calV_3D,
  which yields a value analogous to electric potential energy.
  A higher energy value indicates greater diversity among the group opinions.
  '''

  # Choose the V_max considering group size (N); these values are maximum PE of N-system, respectively.
  V_max_list = np.array([1.4142135623730951, 3.9481594714802353, 7.666977259633668, 12.49614444572656, 18.511218397802732, 25.619490892577677, 33.89610553620064, 43.30531859956281, 53.851507919120344])
  V_max = V_max_list[N-2]

  # Calculate V (mass of external field)
  V = 0
  for i in range(N):
    for j in range(N):
      if i == j:
        continue
      a = S_array[mc,ens,i,:]
      b = S_array[mc,ens,j,:]
      cos_t = np.round(np.dot(a, b),10)
      t = (2-2*cos_t)**0.5
      V += t**0.5

  return 0.5*V/V_max

In [3]:
def run(N, T, alpha, ENS_step, MC_step, CONFIG):
  '''
  N =         No. of agents
  alpha =     Exponent
  ENS_step =  Ensemble
  MC_step =   (Macro) Step
  '''
  S_array = np.zeros((MC_step, ENS_step, N, 3))        # S(t)
  V_array = np.zeros((MC_step, ENS_step))           # V(t)
  #del_S_array = np.zeros((MC_step, ENS_step, N))       # delS(t)

  for mc in range(MC_step):
    for ens in range(ENS_step):

      # 1. Initialize
      if mc == 0:
        for n in range(N):
          theta_dict = {
              'NH': (1/2*np.random.rand())*np.pi,
              'EWH': (1/2*np.random.rand())*np.pi,
              'SH': (1-1/2*np.random.rand())*np.pi
          }
          phi_dict = {
              'NH': (2*np.random.rand())*np.pi,
              'EWH': (2*np.random.rand())*np.pi,
              'SH': (2*np.random.rand())*np.pi
          }
          theta = theta_dict[CONFIG]
          phi = phi_dict[CONFIG]
          if CONFIG == 'EWH':
            S_array[mc,ens,n,:] = np.array([np.sin(theta)*np.cos(phi), -np.cos(theta), np.sin(theta)*np.sin(phi)])
          else:
            S_array[mc,ens,n,:] = np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])
        V_array[mc,ens] = calV_3D(N, ens, mc, S_array)

      # 2. Interact
      else:
        V = V_array[mc-1,ens]
        for i in range(N):
          # 1. Calculate Group Opinion (S_G)
          e = np.array([0.0, 0.0, 1.0])
          v_i = np.array([0.0, 0.0, 0.0])
          for j in range(N):
            si_sj = np.dot(S_array[mc-1,ens,i,:], S_array[mc-1,ens,j,:])
            v_ij = 0.5*(1+si_sj)*S_array[mc-1,ens,j,:]
            v_i += v_ij
          v_i += (N**alpha)*V*e
          v_i_n = np.linalg.norm(v_i)
          v_f = v_i/v_i_n

          # 2. Calculate Prob
          if mc <= T:
            rT = np.sum(S_array[0:mc,ens,i,:], axis=0)
          else:
            rT = np.sum(S_array[mc-T:mc,ens,i,:], axis=0)
          M = np.linalg.norm(rT)
          Th = np.arccos(np.round(np.dot(rT/M, v_f),10))
          Lamb = 1/(1+M*Th)

          # 3. Spin
          p = np.random.rand()
          # 3-1. Change Spins
          if p <= Lamb:
            S_array[mc,ens,i,:] = v_f
          # 3-2. Retain Spins
          else:
            S_array[mc,ens,i,:] = S_array[mc-1,ens,i,:]
          #del_S_array[mc,ens,i] = np.linalg.norm(S_array[mc,ens,i,:] - S_array[mc-1,ens,i,:])
          V_array[mc,ens] = calV_3D(N, ens, mc, S_array)

  return S_array, V_array

# Data generation

The code that generates data for 'FigAll' can be found in another Jupyter notebook (.ipynb file).

## FigIC

In [7]:
N = 4                             # No. of agents
T = 10
alpha = 0.5                       # Exponent
ENS_step = 1000                    # Ensemble
MC_step = 100+1                     # (Macro) Step

spin_config = ['NH', 'EWH', 'SH']

for config in spin_config:
  S_array, V_array = run(N, T, alpha, ENS_step, MC_step, config)

  with open(f"Sarray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
      pickle.dump(S_array, f)
  with open(f"Varray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
      pickle.dump(V_array, f)

## FigT

In [ ]:
N = 4                             # No. of agents
alpha = 0.5                       # Exponent
ENS_step = 1000                    # Ensemble
MC_step = 100+1                     # (Macro) Step

spin_config = ['SH']
T_lst = [1, 10, 100]

for config in spin_config:
    for T in T_lst:
      S_array, V_array = run(N, T, alpha, ENS_step, MC_step, config)

      with open(f"Sarray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
          pickle.dump(S_array, f)
      with open(f"Varray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
          pickle.dump(V_array, f)

## FigTfinal

In [13]:
N = 4                             # No. of agents
alpha = 0.5                       # Exponent
ENS_step = 1000                    # Ensemble
MC_step = 100+1                     # (Macro) Step

spin_config = 'SH'
T_lst = [5,10,15,20,25,30]

for T in T_lst:
  S_array, V_array = run(N, T, alpha, ENS_step, MC_step, config)

  with open(f"Sarray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
      pickle.dump(S_array, f)
  with open(f"Varray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
      pickle.dump(V_array, f)

## Figscale

In [8]:
N = 4                             # No. of agents
T = 2                       # Exponent
ENS_step = 1000                    # Ensemble
MC_step = T*10+1                     # (Macro) Step

config = 'SH'
alpha = 0.5

S_array, V_array = run(N, T, alpha, ENS_step, MC_step, config)

with open(f"Sarray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
  pickle.dump(S_array, f)
with open(f"Varray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
  pickle.dump(V_array, f)

In [ ]:
N = 4                             # No. of agents
T = 10                       # Exponent
ENS_step = 1000                    # Ensemble
MC_step = T*10+1                     # (Macro) Step

config = 'SH'
alpha = 0.5

S_array, V_array = run(N, T, alpha, ENS_step, MC_step, config)

with open(f"Sarray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
  pickle.dump(S_array, f)
with open(f"Varray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
  pickle.dump(V_array, f)

In [23]:
N = 4                             # No. of agents
T = 50                       # Exponent
ENS_step = 1000                    # Ensemble
MC_step = T*10+1                     # (Macro) Step

config = 'SH'
alpha = 0.5

S_array, V_array = run(N, T, alpha, ENS_step, MC_step, config)

with open(f"Sarray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
  pickle.dump(S_array, f)
with open(f"Varray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
  pickle.dump(V_array, f)

In [24]:
N = 4                             # No. of agents
T = 200                       # Exponent
ENS_step = 1000                    # Ensemble
MC_step = T*10+1                     # (Macro) Step

config = 'SH'
alpha = 0.5

S_array, V_array = run(N, T, alpha, ENS_step, MC_step, config)

with open(f"Sarray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
  pickle.dump(S_array, f)
with open(f"Varray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
  pickle.dump(V_array, f)

## FigN

In [14]:
for N in [2,3,4,5,6,7,8,9,10]:
    T = 10
    alpha = 0.5                       # Exponent
    ENS_step = 1000                    # Ensemble
    MC_step = 100+1                     # (Macro) Step

    spin_config = ['NH', 'EWH', 'SH']

    for config in spin_config:
      S_array, V_array = run(N, T, alpha, ENS_step, MC_step, config)
    
      with open(f"Sarray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
          pickle.dump(S_array, f)
      with open(f"Varray_N{N}_T{T}_alpha{alpha}_{config}.pkl","wb") as f:
          pickle.dump(V_array, f)